In [3]:
import MetaTrader5 as mt5
import time
import math
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from forex_python.converter import CurrencyRates
from currency_converter import CurrencyConverter

from random import sample

from tqdm import tqdm
import vectorbt as vbt

from pandas.tseries.offsets import BDay
from itertools import combinations, count

import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import adfuller

from threading import Thread, Lock
from multiprocessing import Process

import requests
import json
import hashlib

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler

In [4]:
colors = cycler('color', ['669FEE', '66EE91', '9988DD', 'EECC55', '88BB44', 'FFBBBB'])
plt.rc('figure', facecolor='313233')
plt.rc('axes', facecolor='313233', edgecolor='none', axisbelow=True, grid=True, prop_cycle=colors, labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor='313233', edgecolor='313233')
plt.rc('text')

In [5]:
def send_telegram_message(message: str, chat_id: str, api_key: str, ):
    responses = {}

    proxies = None
    headers = {'Content-Type': 'application/json',
               'Proxy-Authorization': 'Basic base64'}
    data_dict = {'chat_id': chat_id,
                 'text': message,
                 'parse_mode': 'HTML',
                 'disable_notification': True}
    data = json.dumps(data_dict)
    url = f'https://api.telegram.org/bot{api_key}/sendMessage'

    requests.packages.urllib3.disable_warnings()
    response = requests.post(url,
                             data=data,
                             headers=headers,
                             proxies=proxies,
                             verify=False)
    return response